<a href="https://colab.research.google.com/github/sezerblt/data01/blob/master/keras1_func_api_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(32, 32, 3), name="img")
print("1-",inputs)

x1 = layers.Conv2D(32, 3, activation="relu")(inputs)
print("\nx1    :",x1)
x2 = layers.Conv2D(64, 3, activation="relu")(x1)
print("x2    :",x2)
block_1_output = layers.MaxPooling2D(3)(x2)
print("block1:",block_1_output)

x3 = layers.Conv2D(64, 3, activation="relu", padding="same")(block_1_output)
print("\nx3    :",x3)
x4 = layers.Conv2D(64, 3, activation="relu", padding="same")(x3)
print("x4    :",x4)
block_2_output = layers.add([x4,block_1_output])
print("block2:",block_2_output)

x5 = layers.Conv2D(64, 3, activation="relu", padding="same")(block_2_output)
print("\nx5   :",x5)
x6 = layers.Conv2D(64, 3, activation="relu", padding="same")(x5)
print("x6   :",x6)
block_3_output = layers.add([x6,block_2_output])
print("block3:",block_3_output)

x7 = layers.Conv2D(64, 3, activation="relu")(block_3_output)
print("\nx7   :",x7)
x8 = layers.GlobalAveragePooling2D()(x7)
print("x8   :",x8)

dense = layers.Dense(256, activation="relu")(x8)
print("\ndense:",dense)
dropout = layers.Dropout(0.5)(dense)
print("dropout:",dropout)
outputs=layers.Dense((10))(dropout)
print("outputs:",outputs)

model = keras.Model(inputs, outputs, name="toy_resnet")
print(model)
model.summary()
keras.utils.plot_model(model, "mini_resnet.png", show_shapes=True)

In [0]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
print(x_train.shape,"---",y_train.shape)
print(x_test.shape,"---",y_test.shape)

x_Train = x_train.astype("float32") / 255.0
x_Test =  x_test.astype("float32") / 255.0
y_Train = keras.utils.to_categorical(y_train,10)
y_Test  = keras.utils.to_categorical(y_test,10)
print(y_Train.shape," *** ",y_Test.shape)

model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["acc"],
)

model.fit(x_Train[:1000], y_Train[:1000], batch_size=64, epochs=10, validation_split=0.2)

In [0]:
shared_embedding = layers.Embedding(1000, 128)
text_input_a = keras.Input(shape=(None,), dtype="int32")
text_input_b = keras.Input(shape=(None,), dtype="int32")
encoded_input_a = shared_embedding(text_input_a)
encoded_input_b = shared_embedding(text_input_b)
print(encoded_input_b)

In [0]:
import tensorflow
import numpy as np
vgg19 = tensorflow.keras.applications.VGG19()
features_list = [layer.output for layer in vgg19.layers]
feat_extraction_model = keras.Model(inputs=vgg19.input, outputs=features_list)
img = np.random.random((1, 224, 224, 3)).astype("float32")
extracted_features = feat_extraction_model(img)
print(extracted_features)


In [0]:
import tensorflow as tf
class CustomDense(layers.Layer):
  def __init__(self, units=32):
    super(CustomDense, self).__init__()
    self.units = units

    def build(self, input_shape):
      self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
      self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
      return tf.matmul(inputs, self.w) + self.b
      
    def get_config(self):
      return {"units": self.units}


inputs = keras.Input((4,))
outputs = CustomDense(10)(inputs)

model = keras.Model(inputs, outputs)
config = model.get_config()

new_model = keras.Model.from_config(config, custom_objects={"CustomDense": CustomDense})